In [7]:
import io, contextlib, os
from pytket.circuit.display import render_circuit_jupyter as draw
from pytket.predicates import CompilationUnit
from pytket.qasm import circuit_from_qasm, circuit_to_qasm_str
from pytket.extensions.qiskit import AerBackend
from pathlib import Path
from concurrent.futures import ProcessPoolExecutor


In [5]:
out_dir = Path("../results/tket_results/ibmnew")
out_dir.mkdir(parents=True, exist_ok=True)

def run_circ(str_circ):
    from pytket.extensions.qiskit import AerBackend
    from pytket.qasm import circuit_from_qasm, circuit_to_qasm_str
    from pytket.transform import CompilationUnit
    from pathlib import Path

    opt = AerBackend().default_compilation_pass(optimisation_level=3)
    name = Path(str_circ).stem
    out_file = Path("../results/tket_results/ibmnew") / f"{name}_ibmn_tket.qasm"
    if out_file.exists():
        return
    circ = circuit_from_qasm(str_circ)
    cu = CompilationUnit(circ)
    opt.apply(cu)
    with open(out_file, "w") as f:
        f.write(circuit_to_qasm_str(cu.circuit))
    print(f"Finished {name}")

In [4]:
from pathlib import Path

bench_dir = Path("benchmarks/ibmnew")
out_dir = Path("../results/tket_results/ibmnew")

circs = [
    str(p) for p in bench_dir.rglob("*.qasm")
    if not (out_dir / f"{p.stem}_ibmn_tket.qasm").exists()
]

print(len(circs))

with ProcessPoolExecutor(max_workers=10) as ex:
    ex.map(run_circ, circs)

179
